<a href="https://colab.research.google.com/github/MateoIbaniez/ML---Analisis-de-datos/blob/main/shodan_analisis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
shodan = pd.read_csv('/content/drive/MyDrive/data tp ml/shodan_df_hashed.csv')

In [ ]:
train = pd.read_parquet('/content/drive/MyDrive/data tp ml/train.parq', columns=['attacker_ip_enum', 'label'])

In [ ]:
train

,attacker_ip_enum,label
0,6466,0
1,6466,0
2,4637,0
3,4637,0
4,4637,0
...,...,...
61629680,190784,0
61629681,193446,0
61629682,193446,0
61629683,193446,0


In [ ]:
train.drop_duplicates(inplace=True)

In [ ]:
train

,attacker_ip_enum,label
0,6466,0
2,4637,0
10,8817,0
11,6251,0
13,5510,0
...,...,...
61481746,198862,0
61484942,194893,0
61568580,192953,0
61568584,190292,0


In [ ]:
shodan

,shodan_info,attacker_ip_enum
0,{},5915
1,"{'22/tcp': {'headers_hash': None, 'jarm': None...",3325
2,{},8416
3,{},1213
4,{},9185
...,...,...
197674,"{'1701/udp': {'headers_hash': None, 'jarm': No...",196812
197675,{},191141
197676,"{'80/tcp': {'headers_hash': -282574487, 'jarm'...",195077
197677,{},198002


In [ ]:
shodan.drop_duplicates(inplace=True)

No hay nan, pero si hay valores sin data como '{}'

In [ ]:
shodan.shodan_info.count()

197679

In [ ]:
type(shodan.shodan_info[0])

str

In [ ]:
(shodan.shodan_info[1])

"{'22/tcp': {'headers_hash': None, 'jarm': None, 'ja3s': None}}"

A los '{}' los cambiamos por unknown

In [ ]:
def poner_unknowns(x):
  y = str(x)
  if y == '{}':
    return 'unknown'
  else:
    return y

In [ ]:
shodan.shodan_info = shodan.shodan_info.map(poner_unknowns)

In [ ]:
total = len(shodan)
total_unknown = len(shodan.loc[shodan.shodan_info == 'unknown'])

porcentaje =  total_unknown/total

print("El porcentaje de unknowns en el shodan: " + str(porcentaje) )

El porcentaje de unknowns en el shodan: 0.7592258155899211


In [ ]:
train_shodan = train.merge(shodan)


NameError: ignored

In [ ]:
train_shodan

,attacker_ip_enum,label,shodan_info
0,6466,0,"{'21/tcp': {'headers_hash': None, 'jarm': '000..."
1,4637,0,"{'22/tcp': {'headers_hash': None, 'jarm': None..."
2,8817,0,unknown
3,6251,0,"{'80/tcp': {'headers_hash': -1692967738, 'jarm..."
4,5510,0,"{'21/tcp': {'headers_hash': None, 'jarm': '000..."
...,...,...,...
148254,198862,0,unknown
148255,194893,0,unknown
148256,192953,0,"{'53/tcp': {'headers_hash': None, 'jarm': None..."
148257,190292,0,"{'22/tcp': {'headers_hash': None, 'jarm': None..."


Vemos como si shodan_info es 'unknown' lo mas probable es que sea 0.

Si shodan_info es 'unknown' entonces la proba de que que la label sea 0 es: 0.99947 y de que sea 1 es: 0.0005.

In [ ]:
unknown =  train_shodan.loc[train_shodan.shodan_info == 'unknown']['label']
unknown.value_counts()

0    112396
1        59
Name: label, dtype: int64

In [ ]:
not_unknown = train_shodan.loc[train_shodan.shodan_info != 'unknown']['label']
not_unknown.value_counts()

0    32973
1     2831
Name: label, dtype: int64

Ahora el mismo analisis pero con el data set de 40 millones de columnas de train. Con el obejtivo de ver si esas 59 ips con label 1 con shodan_info en '{}' tiene pocas rows o muchas

In [ ]:
train_full = pd.read_parquet('/content/drive/MyDrive/data tp ml/train.parq')

In [ ]:
train_full.drop(['attacker_as_num', 'watcher_as_num'], axis=1, inplace=True)

In [ ]:
train_full = train_full.drop_duplicates()

In [ ]:
train_shodan_full = train_full.merge(shodan)

In [ ]:
train_shodan_full

,attack_time,watcher_country,watcher_as_name,attacker_country,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,label,shodan_info
0,2023-07-31 07:17:51+00:00,DE,Host Europe GmbH,TR,Murat Aktas,http:exploit,0,6466,0,"{'21/tcp': {'headers_hash': None, 'jarm': '000..."
1,2023-07-31 07:17:51+00:00,DE,Host Europe GmbH,TR,Murat Aktas,http:spam,0,6466,0,"{'21/tcp': {'headers_hash': None, 'jarm': '000..."
2,2023-07-31 07:18:14+00:00,DE,manitu GmbH,TR,Murat Aktas,http:bruteforce,11,6466,0,"{'21/tcp': {'headers_hash': None, 'jarm': '000..."
3,2023-07-31 07:18:14+00:00,DE,manitu GmbH,TR,Murat Aktas,http:spam,11,6466,0,"{'21/tcp': {'headers_hash': None, 'jarm': '000..."
4,2023-07-31 07:18:14+00:00,DE,manitu GmbH,TR,Murat Aktas,http:exploit,11,6466,0,"{'21/tcp': {'headers_hash': None, 'jarm': '000..."
...,...,...,...,...,...,...,...,...,...,...
40290303,2023-08-24 13:20:10+00:00,DE,ENTEGA Medianet GmbH,CN,"ChangZhou Bitcomm Software Technology Co., Lim...",http:scan,24879,199690,0,unknown
40290304,2023-08-24 13:20:14+00:00,DE,ENTEGA Medianet GmbH,CN,"ChangZhou Bitcomm Software Technology Co., Lim...",http:scan,24879,199690,0,unknown
40290305,2023-08-24 13:20:14+00:00,DE,ENTEGA Medianet GmbH,CN,"ChangZhou Bitcomm Software Technology Co., Lim...",http:spam,24879,199690,0,unknown
40290306,2023-08-24 13:20:08+00:00,DE,ENTEGA Medianet GmbH,CN,"ChangZhou Bitcomm Software Technology Co., Lim...",http:spam,24879,199690,0,unknown


In [ ]:
unknown_full =  train_shodan_full.loc[train_shodan_full.shodan_info == 'unknown']['label']
unknown_full.value_counts()

0    12313283
1        5593
Name: label, dtype: int64

In [ ]:
total0 = 12313283
total1 = 5593
total = total0 + total1

porcentaje = total1 / total

print("La proba de que una ip sea de label 1 si su shodan_info es 'unknown' es: " + str(porcentaje) )

La proba de que una ip sea de label 1 si su shodan_info es 'unknown' es: 0.0004540186945627182


In [ ]:
not_unknown_full =  train_shodan_full.loc[train_shodan_full.shodan_info != 'unknown']['label']
not_unknown_full.value_counts()

0    27255484
1      715948
Name: label, dtype: int64

In [ ]:
total0 = 27255484
total1 = 715948
total = total0 + total1

porcentaje = total1 / total

print("La proba de que una ip sea de label 1 si su shodan_info NO es 'unknown' es: " + str(porcentaje) )

La proba de que una ip sea de label 1 si su shodan_info NO es 'unknown' es: 0.025595686341693197


In [ ]:
unk_attCountry = train_shodan_full.loc[train_shodan_full.shodan_info == 'unknown'][ ['label', 'attacker_country'] ]
unk_attCountry

,label,attacker_country
188453,0,GH
188454,0,GH
188455,0,GH
188456,0,GH
188457,0,GH
...,...,...
40290303,0,CN
40290304,0,CN
40290305,0,CN
40290306,0,CN


In [ ]:
unk_attCountry.loc[unk_attCountry.label == 1].attacker_country.value_counts()

US    4103
CA     489
HK     264
TW      62
NG      61
      ... 
GU       0
GW       0
GY       0
HR       0
ZW       0
Name: attacker_country, Length: 214, dtype: int64

In [ ]:
unk_attCountry.loc[unk_attCountry.label == 0].attacker_country.value_counts()

US    4130615
DE    1235744
CN     839854
SG     680310
IN     601525
       ...   
SB          6
WS          0
PF          0
SH          0
IM          0
Name: attacker_country, Length: 214, dtype: int64

In [ ]:
not_unk_attCountry = train_shodan_full.loc[train_shodan_full.shodan_info != 'unknown'][ ['label', 'attacker_country'] ]
not_unk_attCountry

,label,attacker_country
0,0,TR
1,0,TR
2,0,TR
3,0,TR
4,0,TR
...,...,...
40290289,0,SG
40290290,0,SG
40290291,0,SG
40290292,0,SG


In [ ]:
not_unk_attCountry.loc[not_unk_attCountry.label == 1].attacker_country.value_counts().head(10)

US    218152
CA     93030
RU     79131
SG     59827
DE     52799
AU     41917
CN     18194
IN     13865
FR     13642
NL     11045
Name: attacker_country, dtype: int64

In [ ]:
not_unk_attCountry.loc[not_unk_attCountry.label == 0].attacker_country.value_counts().head(10)

US    6685060
DE    2085764
NL    1966303
SG    1728122
FR    1625875
CN    1593285
GB     987600
RU     975309
IN     965833
VN     735642
Name: attacker_country, dtype: int64

In [ ]:
not_unk_country_as = train_shodan_full.loc[train_shodan_full.shodan_info != 'unknown'][ ['shodan_info', 'label', 'attacker_country', 'attacker_as_name', 'watcher_country'] ]


In [ ]:
not_country_as_1 = not_unk_country_as.loc[not_unk_country_as.label == 1][['shodan_info','attacker_country', 'attacker_as_name', 'watcher_country']]
not_country_as_0 = not_unk_country_as.loc[not_unk_country_as.label == 0][['shodan_info','attacker_country', 'attacker_as_name', 'watcher_country']]

not_country_as_1 = not_country_as_1.value_counts()
not_country_as_1 = not_country_as_1.reset_index()
not_country_as_1.columns = ['shodan_info','attacker_country', 'attacker_as_name', 'watcher_country' , 'cantidad_label1' ]

not_country_as_0 = not_country_as_0.value_counts()
not_country_as_0 = not_country_as_0.reset_index()
not_country_as_0.columns = ['shodan_info','attacker_country', 'attacker_as_name', 'watcher_country' , 'cantidad_label0' ]

dicc_not_unk = not_country_as_1.merge(not_country_as_0)

dicc_not_unk['comparacion_bin'] = (dicc_not_unk.cantidad_label1 >= dicc_not_unk.cantidad_label0)
dicc_not_unk.comparacion_bin = dicc_not_unk.comparacion_bin.astype('int8')

dicc_not_unk['comparacion_bin'] = dicc_not_unk['comparacion_bin'] * 2
dicc_not_unk.drop(['cantidad_label1', 'cantidad_label0'], axis=1, inplace=True)

In [ ]:
unk_country_as = train_shodan_full.loc[train_shodan_full.shodan_info == 'unknown'][ ['shodan_info', 'label', 'attacker_country', 'attacker_as_name', 'watcher_country'] ]


In [ ]:
country_as_1 = unk_country_as.loc[unk_country_as.label == 1][['shodan_info','attacker_country', 'attacker_as_name', 'watcher_country']]
country_as_0 = unk_country_as.loc[unk_country_as.label == 0][['shodan_info','attacker_country', 'attacker_as_name', 'watcher_country']]

country_as_1 = country_as_1.value_counts()
country_as_1 = country_as_1.reset_index()
country_as_1.columns = ['shodan_info','attacker_country', 'attacker_as_name', 'watcher_country' , 'cantidad_label1' ]

country_as_0 = country_as_0.value_counts()
country_as_0 = country_as_0.reset_index()
country_as_0.columns = ['shodan_info','attacker_country', 'attacker_as_name', 'watcher_country' , 'cantidad_label0' ]

dicc_unk = country_as_1.merge(country_as_0)

dicc_unk['comparacion_bin'] = (dicc_unk.cantidad_label1 >= dicc_unk.cantidad_label0)
dicc_unk.comparacion_bin = dicc_unk.comparacion_bin.astype('int8')
dicc_unk.drop(['cantidad_label1', 'cantidad_label0'], axis=1, inplace=True)

In [ ]:
dicc = pd.concat([dicc_unk, dicc_not_unk])
dicc

,shodan_info,attacker_country,attacker_as_name,watcher_country,comparacion_bin
0,unknown,US,PERFORMIVE,US,1
1,unknown,US,ASN-CXA-ALL-CCI-22773-RDC,US,0
2,unknown,CA,PERFORMIVE,US,1
3,unknown,HK,HKT Limited,US,0
4,unknown,US,ASN-CXA-ALL-CCI-22773-RDC,DE,0
...,...,...,...,...,...
1535,"{'1701/udp': {'headers_hash': None, 'jarm': No...",CN,Chinanet,DE,0
1536,"{'4444/tcp': {'headers_hash': None, 'jarm': No...",US,AS-SPRIO,LU,2
1537,"{'4444/tcp': {'headers_hash': None, 'jarm': No...",US,24SHELLS,SE,0
1538,"{'2000/tcp': {'headers_hash': None, 'jarm': No...",TZ,Liquid Telecommunications Ltd,PL,0


In [ ]:
train_shodan_full

,attack_time,watcher_country,watcher_as_name,attacker_country,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,label,shodan_info
0,2023-07-31 07:17:51+00:00,DE,Host Europe GmbH,TR,Murat Aktas,http:exploit,0,6466,0,"{'21/tcp': {'headers_hash': None, 'jarm': '000..."
1,2023-07-31 07:17:51+00:00,DE,Host Europe GmbH,TR,Murat Aktas,http:spam,0,6466,0,"{'21/tcp': {'headers_hash': None, 'jarm': '000..."
2,2023-07-31 07:18:14+00:00,DE,manitu GmbH,TR,Murat Aktas,http:bruteforce,11,6466,0,"{'21/tcp': {'headers_hash': None, 'jarm': '000..."
3,2023-07-31 07:18:14+00:00,DE,manitu GmbH,TR,Murat Aktas,http:spam,11,6466,0,"{'21/tcp': {'headers_hash': None, 'jarm': '000..."
4,2023-07-31 07:18:14+00:00,DE,manitu GmbH,TR,Murat Aktas,http:exploit,11,6466,0,"{'21/tcp': {'headers_hash': None, 'jarm': '000..."
...,...,...,...,...,...,...,...,...,...,...
40290303,2023-08-24 13:20:10+00:00,DE,ENTEGA Medianet GmbH,CN,"ChangZhou Bitcomm Software Technology Co., Lim...",http:scan,24879,199690,0,unknown
40290304,2023-08-24 13:20:14+00:00,DE,ENTEGA Medianet GmbH,CN,"ChangZhou Bitcomm Software Technology Co., Lim...",http:scan,24879,199690,0,unknown
40290305,2023-08-24 13:20:14+00:00,DE,ENTEGA Medianet GmbH,CN,"ChangZhou Bitcomm Software Technology Co., Lim...",http:spam,24879,199690,0,unknown
40290306,2023-08-24 13:20:08+00:00,DE,ENTEGA Medianet GmbH,CN,"ChangZhou Bitcomm Software Technology Co., Lim...",http:spam,24879,199690,0,unknown


In [ ]:
aux2 = train_shodan_full.merge(dicc, on=['shodan_info','attacker_country', 'attacker_as_name', 'watcher_country'])
aux2

,attack_time,watcher_country,watcher_as_name,attacker_country,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,label,shodan_info,comparacion_bin
0,2023-08-02 01:28:24+00:00,DE,noris network AG,CN,Shenzhen Tencent Computer Systems Company Limited,ssh:bruteforce,12463,6124,0,"{'22/tcp': {'headers_hash': None, 'jarm': None...",0
1,2023-08-26 00:54:30+00:00,DE,Vereinigte Stadtwerke Media GmbH,CN,Shenzhen Tencent Computer Systems Company Limited,tcp:scan,47388,6124,0,"{'22/tcp': {'headers_hash': None, 'jarm': None...",0
2,2023-08-23 15:11:15+00:00,DE,Contabo GmbH,CN,Shenzhen Tencent Computer Systems Company Limited,ssh:bruteforce,27961,18,0,"{'22/tcp': {'headers_hash': None, 'jarm': None...",0
3,2023-08-23 15:12:15+00:00,DE,Contabo GmbH,CN,Shenzhen Tencent Computer Systems Company Limited,ssh:bruteforce,27961,18,0,"{'22/tcp': {'headers_hash': None, 'jarm': None...",0
4,2023-08-23 15:13:15+00:00,DE,Contabo GmbH,CN,Shenzhen Tencent Computer Systems Company Limited,ssh:bruteforce,27961,18,0,"{'22/tcp': {'headers_hash': None, 'jarm': None...",0
...,...,...,...,...,...,...,...,...,...,...,...
970993,2023-08-13 13:45:17+00:00,FR,OVH SAS,US,LEASEWEB-USA-CHI,http:scan,2063,194898,0,"{'80/tcp': {'headers_hash': 1817912139, 'jarm'...",0
970994,2023-08-12 18:59:21+00:00,FR,OVH SAS,US,LEASEWEB-USA-CHI,http:scan,2063,194898,0,"{'80/tcp': {'headers_hash': 1817912139, 'jarm'...",0
970995,2023-08-12 18:59:21+00:00,FR,OVH SAS,US,LEASEWEB-USA-CHI,http:spam,2063,194898,0,"{'80/tcp': {'headers_hash': 1817912139, 'jarm'...",0
970996,2023-08-09 16:24:56+00:00,FR,Host Europe GmbH,US,LEASEWEB-USA-CHI,http:spam,6893,194898,0,"{'80/tcp': {'headers_hash': 1817912139, 'jarm'...",0
